In [1]:
#Utility functions for ensuring validity of user input, specifically whether the bounding box exists on earth or not
def check_longitude_validity(min_longitude, max_longitude):
    if min_longitude > max_longitude:
        raise Exception("minimum longitude must be less than maximum longitude")
    if min_longitude < -180.0 or min_longitude > 180.0:
        raise Exception("minimum longitude is not a valid value for longitude")
    if max_longitude < -180.0 or max_longitude > 180.0:
        raise Exception("maximum longitude is not a valid value for longitude")


def check_latitude_validity(min_latitude, max_latitude):
    if min_latitude > max_latitude:
        raise Exception("minimum latitude must be less than maximum latitude")
    if min_latitude < -90.0 or min_latitude > 90.0:
        raise Exception("minimum latitude is not a valid value for latitude")
    if max_latitude < -90.0 or max_latitude > 90.0:
        raise Exception("maximum latitude is not a valid value for latitude")


def check_bounding_box_validity(lon_lat_min, lon_lat_max):
    check_longitude_validity(min_longitude=lon_lat_min[0], max_longitude=lon_lat_max[0])
    check_latitude_validity(min_latitude=lon_lat_min[1], max_latitude=lon_lat_max[1])
    


In [2]:
#class for organizing all user provided options for rtc-gamma and composite process.
#In the original script this is done through command line options

from getpass import getpass

class compositeOptions():
    def init(self):
        return None
    def get_options(self):
        self.earthdata_username=input("provide your earthdata username: ")
        self.earthdata_password=getpass("provide your earthdata password: ")
        self.start_date=input("provide start date for granule search in the form of yyyy-mm-dd or leave blank: ")
        self.end_date=input("provide end date for granule search in the form of yyyy-mm-dd or leave blank: ")
        self.lon_lat_minimum=input("provide a coordinate to define lower left corner of bounding box in the form 'min_lon min_lat': ")
        self.lon_lat_maximum=input("provide a coordinate to define upper right corner of bounding box in the form 'max_lon max_lat': ")
        self.granule_count=int(input("provide the maximum number of granules for composite process: "))
        self.polarization=input("provide desired polarization of granules [vv, vh, vv+vh]") 
        
        self.parse_bounding_box()
    
    def parse_bounding_box(self):
        self.lon_lat_minimum = self.lon_lat_minimum.split(" ")
        self.lon_lat_minimum[0] = float(self.lon_lat_minimum[0])
        self.lon_lat_minimum[1] = float(self.lon_lat_minimum[1])
        
        self.lon_lat_maximum = self.lon_lat_maximum.split(" ")
        self.lon_lat_maximum[0] = float(self.lon_lat_maximum[0])
        self.lon_lat_maximum[1] = float(self.lon_lat_maximum[1])
        
        check_bounding_box_validity(self.lon_lat_minimum, self.lon_lat_maximum)

In [17]:
#Prompt user to provide input and create object holding that input for duration of processing 
composite_options=compositeOptions()
composite_options.get_options()

provide your earthdata username: hadimarchi
provide your earthdata password: ········
provide start date for granule search in the form of yyyy-mm-dd or leave blank: 2012-05-20
provide end date for granule search in the form of yyyy-mm-dd or leave blank: 2016-05-20
provide a coordinate to define lower left corner of bounding box in the form 'min_lon min_lat': 120.0 50.0
provide a coordinate to define upper right corner of bounding box in the form 'max_lon max_lat': 122.0 53.0
provide the maximum number of granules for composite process: 6
provide desired polarization of granules [vv, vh, vv+vh]vv+vh


In [18]:
#Main utility functions for buidling earthdata search url

def get_fetch_url_bbox(composite_options):
    min_bbox_bound = composite_options.lon_lat_minimum
    max_bbox_bound = composite_options.lon_lat_maximum

    fetch_url_bbox = "bbox={min_lon},{min_lat},{max_lon},{max_lat}&".format(
        min_lon=min_bbox_bound[0],
        min_lat=min_bbox_bound[1],
        max_lon=max_bbox_bound[0],
        max_lat=max_bbox_bound[1]
        )
    return fetch_url_bbox


def get_fetch_url_date_range(composite_options):
    fetch_url_date_range = "start={start_date}T00:00:00UTC&end={end_date}T00:00:00&".format(
        start_date=composite_options.start_date if composite_options.start_date is not None else "2014-06-15",
        end_date=composite_options.end_date if composite_options.end_date is not None else str(date.today())
        )
    return fetch_url_date_range


def get_fetch_url_polarization(composite_options):
    fetch_url_polarization = "polarization={polarization}".format(
        polarization=composite_options.polarization.upper()
    ).replace("+", "%2B")
    return fetch_url_polarization


def get_granule_names(granule_list):
    granule_names = []
    for granule in granule_list:
        name = granule["granuleName"]
        granule_names.append(name)
    return granule_names

In [19]:
# Search url for granules to process with rtc gamma

def build_search_url(composite_options):
    base_url = "https://api.daac.asf.alaska.edu/services/search/param?"

    fetch_url_bbox = get_fetch_url_bbox(composite_options)

    fetch_url_date_range = get_fetch_url_date_range(composite_options)

    fetch_url_polarization = get_fetch_url_polarization(composite_options)

    fetch_url = "{base_url}{bbox}{date_range}{polarization}&platform=S1&processingLevel=GRD_HD&output=JSON".format(
        base_url=base_url,
        bbox=fetch_url_bbox,
        date_range=fetch_url_date_range,
        polarization=fetch_url_polarization
        )
    return fetch_url

In [20]:
import requests

def fetch_granules(composite_options):
    search_url = build_search_url(composite_options)
    granules = requests.get(search_url).json()[0] 
    # Somewhat confusing format but .json() call creates a list where the first element
    # is a dictionary containing a variety of information about said granules
    granule_names = get_granule_names(granules)
    granule_names = granule_names[0:composite_options.granule_count]

    if granule_names == []:
        raise Exception("No granules found")
    return granule_names

In [21]:
granules = fetch_granules(composite_options=composite_options)

In [22]:
from hyp3_sdk import HyP3
from datetime import datetime

def rtc_gamma(granules, composite_options):
    hyp3 = HyP3(username=composite_options.earthdata_username, password=composite_options.earthdata_password)
    rtc_gamma_jobs = [hyp3.prepare_rtc_job(granule=granule, include_scattering_area=True) for granule in granules]
    rtc_gamma_batch = hyp3.submit_prepared_jobs(rtc_gamma_jobs)
    rtc_gamma_batch = hyp3.watch(rtc_gamma_batch)
    return rtc_gamma_batch

def download_gamma_products(gamma_batch, composite_options):
    
    download_location = "{now}_min:{lon_lat_min}_max:{lon_lat_max}_granules:{granule_count}".format(
        now=datetime.now().isoformat(),
        lon_lat_min=composite_options.lon_lat_minimum,
        lon_lat_max=composite_options.lon_lat_maximum,
        granule_count=len(granules)
    )
    gamma_batch.download_files(location=download_location)
    return download_location

def run_rtc_gamma(granules, composite_options):
    rtc_gamma_batch = rtc_gamma(granules, composite_options)
    download_location = download_gamma_products(gamma_batch=rtc_gamma_batch, composite_options=composite_options)
    return download_location

In [23]:
download_location = run_rtc_gamma(granules=granules, composite_options=composite_options)

  0%|          | 0/6 [timeout in 10800 s]

  0%|          | 0/6 [00:00<?, ?it/s]

S1A_IW_20160518T221746_DVP_RTC30_G_gpuned_7CA9.zip:   0%|          | 0/665038682 [00:00<?, ?it/s]

S1A_IW_20160518T221721_DVP_RTC30_G_gpuned_8C28.zip:   0%|          | 0/672851469 [00:00<?, ?it/s]

S1A_IW_20160518T221652_DVP_RTC30_G_gpuned_4EDB.zip:   0%|          | 0/759921477 [00:00<?, ?it/s]

S1A_IW_20160513T220936_DVP_RTC30_G_gpuned_FC36.zip:   0%|          | 0/671055216 [00:00<?, ?it/s]

S1A_IW_20160513T220911_DVP_RTC30_G_gpuned_9DCC.zip:   0%|          | 0/673242189 [00:00<?, ?it/s]

S1A_IW_20160513T220842_DVP_RTC30_G_gpuned_B653.zip:   0%|          | 0/762373157 [00:00<?, ?it/s]

In [24]:
# This cell includes functions for running the make_composite tool in the asf_tools package on
# the rtc gamma products we have just generated and downloaded

from zipfile import ZipFile
from pathlib import Path
from asf_tools.composite import make_composite


def collect_polarized_products(products_path, polarization):
    products = list(products_path.glob("**/*{polarization}.tif".format(
        polarization=polarization)
        ))
    products = [str(product) for product in products]
    return products

def unzip_rtc_products(download_location, polarization):
    products_path = Path(download_location)
    zipped_products = list(products_path.glob('**/*.zip'))
    for zipped_product in zipped_products:
        with ZipFile(zipped_product, 'r') as zipObj:
            zipObj.extractall("{location}".format(location=download_location))
    unzipped_products = {"vv_products": collect_polarized_products(products_path, "VV") if "vv" in polarization else [],
                         "vh_products": collect_polarized_products(products_path, "VH") if "vh" in polarization else []} 
    return unzipped_products


def run_make_composite(download_location, composite_options):
    unzipped_products = unzip_rtc_products(download_location, polarization=composite_options.polarization)
    if unzipped_products["vv_products"] != []:
        make_composite("{location}/VV-composite".format(location=download_location),
                       unzipped_products["vv_products"])
    if unzipped_products["vh_products"] != []:
        make_composite("{location}/VH-composite".format(location=download_location),
                       unzipped_products["vh_products"])

In [25]:
# Running make_composite

run_make_composite(download_location=download_location, composite_options=composite_options)